In [28]:
import os
import ffmpeg
import numpy as np
import cv2

# Create a directory to store the frames
output_image_dir = "output_frames"
if os.path.exists(output_image_dir):
    for file in os.listdir(output_image_dir):
        file_path = os.path.join(output_image_dir, file)
        if os.path.isfile(file_path):
            os.unlink(file_path)
os.makedirs(output_image_dir, exist_ok=True)

# Function to find the two strongest horizontal lines in a single frame
def find_two_strongest_horizontal_lines(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)  # Apply Gaussian blur before edge detection
    edges = cv2.Canny(blurred, 30, 100, apertureSize=3)  # Lowered thresholds for more sensitivity
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)  # Reduced threshold for more lines

    top = 0
    bottom = frame.shape[0]

    if lines is not None:
        strongest_lines = []
        for rho, theta in lines[:, 0]:
            theta_diff = abs(theta - np.pi / 2)
            strongest_lines.append((theta_diff, rho, theta))

        strongest_lines = sorted(strongest_lines, key=lambda x: x[0])[:2]

        if len(strongest_lines) == 2:
            y_coords = []
            for _, rho, theta in strongest_lines:
                a = np.cos(theta)
                b = np.sin(theta)
                y0 = b * rho
                y_coords.append(int(y0))

            y_coords.sort()
            if len(y_coords) == 2 and y_coords[0] < y_coords[1]:
                top = max(0, y_coords[0] - 10)
                bottom = min(frame.shape[0], y_coords[1] + 10)

    return top, bottom

# Function to segment white and black keys using binary masks
def segment_piano_keys(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Threshold to create a binary mask for white keys
    _, white_key_mask = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)

    # Threshold to create a binary mask for black keys (invert white keys mask)
    black_key_mask = cv2.bitwise_not(white_key_mask)

    return white_key_mask, black_key_mask

# Function for seeded segmentation based on black key positions
def seeded_segmentation(black_key_mask, frame):
    # Find contours in the black key mask
    contours, _ = cv2.findContours(black_key_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    seeds = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Center of the black key
        seed_center = (x + w // 2, y + h // 2)

        # Ensure the seed is within the contour
        if cv2.pointPolygonTest(contour, seed_center, False) >= 0:
            seeds.append(seed_center)

        # Left and right of the black key
        seed_left = (x, y + h // 2)
        seed_right = (x + w, y + h // 2)

        if cv2.pointPolygonTest(contour, seed_left, False) >= 0:
            seeds.append(seed_left)
        if cv2.pointPolygonTest(contour, seed_right, False) >= 0:
            seeds.append(seed_right)

    # Visualize seeds on the frame
    for seed in seeds:
        cv2.circle(frame, seed, 5, (0, 255, 0), -1)  # Draw seeds as green circles

    # Create a labeled image for seeded segmentation
    labeled_mask = np.zeros_like(black_key_mask, dtype=np.int32)
    for i, seed in enumerate(seeds):
        if 0 <= seed[0] < black_key_mask.shape[1] and 0 <= seed[1] < black_key_mask.shape[0]:
            labeled_mask[seed[1], seed[0]] = i + 1

    # Perform watershed algorithm for segmentation
    gray = cv2.cvtColor(black_key_mask, cv2.COLOR_GRAY2BGR)
    distance_transform = cv2.distanceTransform(black_key_mask, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(distance_transform, 0.9 * distance_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(black_key_mask, sure_fg)

    markers = cv2.connectedComponents(sure_fg)[1]
    markers = markers + 1
    markers[unknown == 255] = 0

    segmented = cv2.watershed(gray, markers)

    # Overlay the segmented regions on the original frame
    frame[segmented == -1] = [0, 0, 255]  # Mark boundaries in red

    return frame

# Input video file
input_video_path = "dataset/MIDItest/miditest_videos/5.mp4"

# Probe video dimensions
probe = ffmpeg.probe(input_video_path)
video_stream = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
width = int(video_stream['width'])
height = int(video_stream['height'])

# Read the first frame to determine cropping boundaries
process = (
    ffmpeg.input(input_video_path)
    .output('pipe:', format='rawvideo', pix_fmt='bgr24')
    .run_async(pipe_stdout=True)
)

in_bytes = process.stdout.read(width * height * 3)
frame = np.frombuffer(in_bytes, np.uint8).reshape([height, width, 3])

top, bottom = find_two_strongest_horizontal_lines(frame)
process.stdout.close()
process.wait()

# Read video frames again and save each frame as an image
process = (
    ffmpeg.input(input_video_path)
    .output('pipe:', format='rawvideo', pix_fmt='bgr24')
    .run_async(pipe_stdout=True)
)

frame_count = 0

while True:
    in_bytes = process.stdout.read(width * height * 3)
    if not in_bytes:
        break

    frame = np.frombuffer(in_bytes, np.uint8).reshape([height, width, 3])

    # Crop the frame
    cropped_frame = frame[top:bottom, :]
    
    # Create a writable copy of the cropped frame
    cropped_frame = cropped_frame.copy()

    # Segment piano keys in the cropped frame
    white_key_mask, black_key_mask = segment_piano_keys(cropped_frame)

    # Perform seeded segmentation
    segmented_frame = seeded_segmentation(black_key_mask, cropped_frame)

    # Save the segmented frame
    segmented_path = os.path.join(output_image_dir, f"segmented_frame_{frame_count:04d}.jpg")
    cv2.imwrite(segmented_path, segmented_frame)

    frame_count += 1
    print(f"Saved {segmented_path}")

# Cleanup
process.stdout.close()
process.wait()
cv2.destroyAllWindows()


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developersrate=231355.5kbits/s speed=0.000455x    
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --ena

Saved output_frames/segmented_frame_0000.jpg
Saved output_frames/segmented_frame_0001.jpg
Saved output_frames/segmented_frame_0002.jpg
Saved output_frames/segmented_frame_0003.jpg
Saved output_frames/segmented_frame_0004.jpg
Saved output_frames/segmented_frame_0005.jpg
Saved output_frames/segmented_frame_0006.jpg
Saved output_frames/segmented_frame_0007.jpg
Saved output_frames/segmented_frame_0008.jpg
Saved output_frames/segmented_frame_0009.jpg
Saved output_frames/segmented_frame_0010.jpg
Saved output_frames/segmented_frame_0011.jpg
Saved output_frames/segmented_frame_0012.jpg
Saved output_frames/segmented_frame_0013.jpg
Saved output_frames/segmented_frame_0014.jpg
Saved output_frames/segmented_frame_0015.jpg
Saved output_frames/segmented_frame_0016.jpg
Saved output_frames/segmented_frame_0017.jpg
Saved output_frames/segmented_frame_0018.jpg
Saved output_frames/segmented_frame_0019.jpg
Saved output_frames/segmented_frame_0020.jpg


Saved output_frames/segmented_frame_0021.jpg
Saved output_frames/segmented_frame_0022.jpg
Saved output_frames/segmented_frame_0023.jpg
Saved output_frames/segmented_frame_0024.jpg
Saved output_frames/segmented_frame_0025.jpg
Saved output_frames/segmented_frame_0026.jpg
Saved output_frames/segmented_frame_0027.jpg
Saved output_frames/segmented_frame_0028.jpg
Saved output_frames/segmented_frame_0029.jpg
Saved output_frames/segmented_frame_0030.jpg
Saved output_frames/segmented_frame_0031.jpg
Saved output_frames/segmented_frame_0032.jpg
Saved output_frames/segmented_frame_0033.jpg
Saved output_frames/segmented_frame_0034.jpg
Saved output_frames/segmented_frame_0035.jpg
Saved output_frames/segmented_frame_0036.jpg
Saved output_frames/segmented_frame_0037.jpg


Saved output_frames/segmented_frame_0038.jpg
Saved output_frames/segmented_frame_0039.jpg
Saved output_frames/segmented_frame_0040.jpg
Saved output_frames/segmented_frame_0041.jpg
Saved output_frames/segmented_frame_0042.jpg
Saved output_frames/segmented_frame_0043.jpg
Saved output_frames/segmented_frame_0044.jpg
Saved output_frames/segmented_frame_0045.jpg
Saved output_frames/segmented_frame_0046.jpg
Saved output_frames/segmented_frame_0047.jpg
Saved output_frames/segmented_frame_0048.jpg
Saved output_frames/segmented_frame_0049.jpg
Saved output_frames/segmented_frame_0050.jpg
Saved output_frames/segmented_frame_0051.jpg
Saved output_frames/segmented_frame_0052.jpg
Saved output_frames/segmented_frame_0053.jpg
Saved output_frames/segmented_frame_0054.jpg
Saved output_frames/segmented_frame_0055.jpg
Saved output_frames/segmented_frame_0056.jpg
Saved output_frames/segmented_frame_0057.jpg
Saved output_frames/segmented_frame_0058.jpg
Saved output_frames/segmented_frame_0059.jpg
Saved outp

Saved output_frames/segmented_frame_0065.jpg
Saved output_frames/segmented_frame_0066.jpg
Saved output_frames/segmented_frame_0067.jpg
Saved output_frames/segmented_frame_0068.jpg
Saved output_frames/segmented_frame_0069.jpg
Saved output_frames/segmented_frame_0070.jpg
Saved output_frames/segmented_frame_0071.jpg
Saved output_frames/segmented_frame_0072.jpg
Saved output_frames/segmented_frame_0073.jpg
Saved output_frames/segmented_frame_0074.jpg
Saved output_frames/segmented_frame_0075.jpg
Saved output_frames/segmented_frame_0076.jpg
Saved output_frames/segmented_frame_0077.jpg


...

Saved output_frames/segmented_frame_3615.jpg
Saved output_frames/segmented_frame_3616.jpg
Saved output_frames/segmented_frame_3617.jpg
Saved output_frames/segmented_frame_3618.jpg
Saved output_frames/segmented_frame_3619.jpg
Saved output_frames/segmented_frame_3620.jpg
Saved output_frames/segmented_frame_3621.jpg
Saved output_frames/segmented_frame_3622.jpg
Saved output_frames/segmented_frame_3623.jpg
Saved output_frames/segmented_frame_3624.jpg
Saved output_frames/segmented_frame_3625.jpg
Saved output_frames/segmented_frame_3626.jpg
Saved output_frames/segmented_frame_3627.jpg
Saved output_frames/segmented_frame_3628.jpg
Saved output_frames/segmented_frame_3629.jpg
Saved output_frames/segmented_frame_3630.jpg
Saved output_frames/segmented_frame_3631.jpg
Saved output_frames/segmented_frame_3632.jpg
Saved output_frames/segmented_frame_3633.jpg
Saved output_frames/segmented_frame_3634.jpg
Saved output_frames/segmented_frame_3635.jpg
Saved output_frames/segmented_frame_3636.jpg
Saved outp

Saved output_frames/segmented_frame_3650.jpg
Saved output_frames/segmented_frame_3651.jpg
Saved output_frames/segmented_frame_3652.jpg
Saved output_frames/segmented_frame_3653.jpg
Saved output_frames/segmented_frame_3654.jpg
Saved output_frames/segmented_frame_3655.jpg
Saved output_frames/segmented_frame_3656.jpg
Saved output_frames/segmented_frame_3657.jpg
Saved output_frames/segmented_frame_3658.jpg
Saved output_frames/segmented_frame_3659.jpg
Saved output_frames/segmented_frame_3660.jpg
Saved output_frames/segmented_frame_3661.jpg
Saved output_frames/segmented_frame_3662.jpg
Saved output_frames/segmented_frame_3663.jpg
Saved output_frames/segmented_frame_3664.jpg
Saved output_frames/segmented_frame_3665.jpg
Saved output_frames/segmented_frame_3666.jpg
Saved output_frames/segmented_frame_3667.jpg
Saved output_frames/segmented_frame_3668.jpg
Saved output_frames/segmented_frame_3669.jpg
Saved output_frames/segmented_frame_3670.jpg
Saved output_frames/segmented_frame_3671.jpg
Saved outp

Saved output_frames/segmented_frame_3674.jpg
Saved output_frames/segmented_frame_3675.jpg
Saved output_frames/segmented_frame_3676.jpg
Saved output_frames/segmented_frame_3677.jpg
Saved output_frames/segmented_frame_3678.jpg
Saved output_frames/segmented_frame_3679.jpg
Saved output_frames/segmented_frame_3680.jpg
Saved output_frames/segmented_frame_3681.jpg
Saved output_frames/segmented_frame_3682.jpg
Saved output_frames/segmented_frame_3683.jpg
Saved output_frames/segmented_frame_3684.jpg
Saved output_frames/segmented_frame_3685.jpg
Saved output_frames/segmented_frame_3686.jpg
Saved output_frames/segmented_frame_3687.jpg
Saved output_frames/segmented_frame_3688.jpg
Saved output_frames/segmented_frame_3689.jpg
Saved output_frames/segmented_frame_3690.jpg
Saved output_frames/segmented_frame_3691.jpg
Saved output_frames/segmented_frame_3692.jpg
Saved output_frames/segmented_frame_3693.jpg
Saved output_frames/segmented_frame_3694.jpg
Saved output_frames/segmented_frame_3695.jpg
Saved outp

Saved output_frames/segmented_frame_3709.jpg
Saved output_frames/segmented_frame_3710.jpg
Saved output_frames/segmented_frame_3711.jpg
Saved output_frames/segmented_frame_3712.jpg
Saved output_frames/segmented_frame_3713.jpg
Saved output_frames/segmented_frame_3714.jpg
Saved output_frames/segmented_frame_3715.jpg
Saved output_frames/segmented_frame_3716.jpg
Saved output_frames/segmented_frame_3717.jpg
Saved output_frames/segmented_frame_3718.jpg
Saved output_frames/segmented_frame_3719.jpg
Saved output_frames/segmented_frame_3720.jpg
Saved output_frames/segmented_frame_3721.jpg
Saved output_frames/segmented_frame_3722.jpg
Saved output_frames/segmented_frame_3723.jpg
Saved output_frames/segmented_frame_3724.jpg
Saved output_frames/segmented_frame_3725.jpg
Saved output_frames/segmented_frame_3726.jpg
Saved output_frames/segmented_frame_3727.jpg
Saved output_frames/segmented_frame_3728.jpg
Saved output_frames/segmented_frame_3729.jpg
Saved output_frames/segmented_frame_3730.jpg
Saved outp

[out#0/rawvideo @ 0x7fc3c79044c0] video:3753787KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.000000%
frame= 3741 fps= 48 q=-0.0 Lsize= 3753787KiB time=00:02:06.00 bitrate=244051.8kbits/s speed=1.63x    


In [29]:
import pretty_midi
import numpy as np
import json

def midi_to_piano_vector(file_path):
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(file_path)

    # Define the range of piano notes (A0 to C8 corresponds to MIDI notes 21 to 108)
    piano_range = range(21, 109)

    # Collect all note events
    note_events = []
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            # Normalize the velocity to the range [0, 1]
            normalized_velocity = note.velocity / 127.0
            note_name = pretty_midi.note_number_to_name(note.pitch)
            note_events.append((note.start, note.pitch, normalized_velocity, 'on', note_name))  # Note-on event
            note_events.append((note.end, note.pitch, normalized_velocity, 'off', note_name))  # Note-off event

    # Sort events by time
    note_events.sort(key=lambda x: x[0])

    # Create a list to store the output
    piano_vectors = []
 
    # Initialize the state of the piano (88-dimensional vector)
    piano_state = np.zeros(88, dtype=float)

    # Process each event
    for event in note_events:
        timestamp, pitch, velocity, event_type, note_name = event

        if pitch in piano_range:
            note_index = pitch - 21  # Map pitch to the 88-key piano range

            if event_type == 'on':
                piano_state[note_index] = velocity  # Set the normalized velocity
            elif event_type == 'off':
                piano_state[note_index] = 0  # Release the note

        # Shift timestamp by 0.6 seconds
        shifted_timestamp = timestamp - 0.25

        # Append the current state, shifted timestamp, and note name
        piano_vectors.append({"timestamp": shifted_timestamp, "vector": piano_state.tolist(), "note_name": note_name})

    return piano_vectors, midi_data.get_end_time()

# Example usage
def save_output_to_file(midi_file, output_file):
    output, midi_duration = midi_to_piano_vector(midi_file)
    
    # Include the total duration in the output
    result = {
        "duration": midi_duration,
        "piano_vectors": output
    }

    # Output the result to a JSON file
    with open(output_file, 'w') as f:
        json.dump(result, f, indent=4)

# Replace with your MIDI file path and desired output file path
midi_file = '/Users/tunaonat/Desktop/proje-git/piano-transcription/dataset/MIDItest/miditest_MIDI/5.mid'
output_file = 'output.json'
save_output_to_file(midi_file, output_file)


In [30]:
import os
import ffmpeg
import numpy as np
import cv2

# Create a directory to store the difference frames
output_diff_dir = "output_differences"
if os.path.exists(output_diff_dir):
    for file in os.listdir(output_diff_dir):
        file_path = os.path.join(output_diff_dir, file)
        if os.path.isfile(file_path):
            os.unlink(file_path)
os.makedirs(output_diff_dir, exist_ok=True)

# Function to find the two strongest horizontal lines in a single frame
def find_two_strongest_horizontal_lines(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 30, 100, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

    top = 0
    bottom = frame.shape[0]

    if lines is not None:
        strongest_lines = []
        for rho, theta in lines[:, 0]:
            theta_diff = abs(theta - np.pi / 2)
            strongest_lines.append((theta_diff, rho, theta))

        strongest_lines = sorted(strongest_lines, key=lambda x: x[0])[:2]

        if len(strongest_lines) == 2:
            y_coords = []
            for _, rho, theta in strongest_lines:
                a = np.cos(theta)
                b = np.sin(theta)
                y0 = b * rho
                y_coords.append(int(y0))

            y_coords.sort()
            if len(y_coords) == 2 and y_coords[0] < y_coords[1]:
                top = max(0, y_coords[0] - 10)
                bottom = min(frame.shape[0], y_coords[1] + 10)

    return top, bottom

# Input video file
input_video_path = "/Users/tunaonat/Desktop/proje-git/piano-transcription/dataset/MIDItest/miditest_videos/5.mp4"

# Probe video dimensions
probe = ffmpeg.probe(input_video_path)
video_stream = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
width = int(video_stream['width'])
height = int(video_stream['height'])
fps = eval(video_stream['r_frame_rate'])  # Get FPS as a float

# Read the first frame to determine cropping boundaries
process = (
    ffmpeg.input(input_video_path)
    .output('pipe:', format='rawvideo', pix_fmt='bgr24')
    .run_async(pipe_stdout=True)
)

in_bytes = process.stdout.read(width * height * 3)
frame = np.frombuffer(in_bytes, np.uint8).reshape([height, width, 3])

top, bottom = find_two_strongest_horizontal_lines(frame)
process.stdout.close()
process.wait()

# Read video frames again and calculate differences
process = (
    ffmpeg.input(input_video_path)
    .output('pipe:', format='rawvideo', pix_fmt='bgr24')
    .run_async(pipe_stdout=True)
)

prev_frame = None
frame_count = 0

while True:
    in_bytes = process.stdout.read(width * height * 3)
    if not in_bytes:
        break

    frame = np.frombuffer(in_bytes, np.uint8).reshape([height, width, 3])

    # Crop the frame
    cropped_frame = frame[top:bottom, :]

    # Convert to grayscale for difference calculation
    gray_frame = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2GRAY)

    if prev_frame is not None:
        # Compute absolute difference
        diff = cv2.absdiff(prev_frame, gray_frame)

        # Generate timestamp
        timestamp = frame_count / fps
        timestamp_text = f"{timestamp:.2f}s"

        # Save the difference frame with timestamp in the filename
        diff_path = os.path.join(output_diff_dir, f"frame_diff_{timestamp_text}_{frame_count:04d}.jpg")
        cv2.imwrite(diff_path, diff)

        print(f"Saved {diff_path}")

    # Update previous frame
    prev_frame = gray_frame
    frame_count += 1

# Cleanup
process.stdout.close()
process.wait()
cv2.destroyAllWindows()


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developersrate=231355.5kbits/s speed=0.000452x    
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --ena

Saved output_differences/frame_diff_0.03s_0001.jpg
Saved output_differences/frame_diff_0.07s_0002.jpg
Saved output_differences/frame_diff_0.10s_0003.jpg
Saved output_differences/frame_diff_0.13s_0004.jpg
Saved output_differences/frame_diff_0.17s_0005.jpg
Saved output_differences/frame_diff_0.20s_0006.jpg
Saved output_differences/frame_diff_0.24s_0007.jpg
Saved output_differences/frame_diff_0.27s_0008.jpg
Saved output_differences/frame_diff_0.30s_0009.jpg
Saved output_differences/frame_diff_0.34s_0010.jpg
Saved output_differences/frame_diff_0.37s_0011.jpg
Saved output_differences/frame_diff_0.40s_0012.jpg
Saved output_differences/frame_diff_0.44s_0013.jpg
Saved output_differences/frame_diff_0.47s_0014.jpg
Saved output_differences/frame_diff_0.51s_0015.jpg
Saved output_differences/frame_diff_0.54s_0016.jpg
Saved output_differences/frame_diff_0.57s_0017.jpg
Saved output_differences/frame_diff_0.61s_0018.jpg
Saved output_differences/frame_diff_0.64s_0019.jpg
Saved output_differences/frame_

Saved output_differences/frame_diff_2.53s_0075.jpg
Saved output_differences/frame_diff_2.56s_0076.jpg
Saved output_differences/frame_diff_2.59s_0077.jpg
Saved output_differences/frame_diff_2.63s_0078.jpg
Saved output_differences/frame_diff_2.66s_0079.jpg
Saved output_differences/frame_diff_2.69s_0080.jpg
Saved output_differences/frame_diff_2.73s_0081.jpg
Saved output_differences/frame_diff_2.76s_0082.jpg
Saved output_differences/frame_diff_2.80s_0083.jpg
Saved output_differences/frame_diff_2.83s_0084.jpg
Saved output_differences/frame_diff_2.86s_0085.jpg
Saved output_differences/frame_diff_2.90s_0086.jpg
Saved output_differences/frame_diff_2.93s_0087.jpg
Saved output_differences/frame_diff_2.96s_0088.jpg
Saved output_differences/frame_diff_3.00s_0089.jpg
Saved output_differences/frame_diff_3.03s_0090.jpg
Saved output_differences/frame_diff_3.07s_0091.jpg
Saved output_differences/frame_diff_3.10s_0092.jpg
Saved output_differences/frame_diff_3.13s_0093.jpg
Saved output_differences/frame_

Saved output_differences/frame_diff_7.14s_0212.jpg
Saved output_differences/frame_diff_7.17s_0213.jpg
Saved output_differences/frame_diff_7.21s_0214.jpg
Saved output_differences/frame_diff_7.24s_0215.jpg
Saved output_differences/frame_diff_7.28s_0216.jpg
Saved output_differences/frame_diff_7.31s_0217.jpg
Saved output_differences/frame_diff_7.34s_0218.jpg
Saved output_differences/frame_diff_7.38s_0219.jpg
Saved output_differences/frame_diff_7.41s_0220.jpg
Saved output_differences/frame_diff_7.44s_0221.jpg
Saved output_differences/frame_diff_7.48s_0222.jpg
Saved output_differences/frame_diff_7.51s_0223.jpg
Saved output_differences/frame_diff_7.54s_0224.jpg
Saved output_differences/frame_diff_7.58s_0225.jpg
Saved output_differences/frame_diff_7.61s_0226.jpg
Saved output_differences/frame_diff_7.65s_0227.jpg
Saved output_differences/frame_diff_7.68s_0228.jpg
Saved output_differences/frame_diff_7.71s_0229.jpg
Saved output_differences/frame_diff_7.75s_0230.jpg
Saved output_differences/frame_

Saved output_differences/frame_diff_9.94s_0295.jpg
Saved output_differences/frame_diff_9.97s_0296.jpg
Saved output_differences/frame_diff_10.00s_0297.jpg
Saved output_differences/frame_diff_10.04s_0298.jpg
Saved output_differences/frame_diff_10.07s_0299.jpg
Saved output_differences/frame_diff_10.10s_0300.jpg
Saved output_differences/frame_diff_10.14s_0301.jpg
Saved output_differences/frame_diff_10.17s_0302.jpg
Saved output_differences/frame_diff_10.21s_0303.jpg
Saved output_differences/frame_diff_10.24s_0304.jpg
Saved output_differences/frame_diff_10.27s_0305.jpg
Saved output_differences/frame_diff_10.31s_0306.jpg
Saved output_differences/frame_diff_10.34s_0307.jpg
Saved output_differences/frame_diff_10.37s_0308.jpg
Saved output_differences/frame_diff_10.41s_0309.jpg
Saved output_differences/frame_diff_10.44s_0310.jpg
Saved output_differences/frame_diff_10.47s_0311.jpg
Saved output_differences/frame_diff_10.51s_0312.jpg
Saved output_differences/frame_diff_10.54s_0313.jpg
Saved output_d

...

Saved output_differences/frame_diff_111.32s_3305.jpg
Saved output_differences/frame_diff_111.35s_3306.jpg
Saved output_differences/frame_diff_111.38s_3307.jpg
Saved output_differences/frame_diff_111.42s_3308.jpg
Saved output_differences/frame_diff_111.45s_3309.jpg
Saved output_differences/frame_diff_111.49s_3310.jpg
Saved output_differences/frame_diff_111.52s_3311.jpg
Saved output_differences/frame_diff_111.55s_3312.jpg
Saved output_differences/frame_diff_111.59s_3313.jpg
Saved output_differences/frame_diff_111.62s_3314.jpg
Saved output_differences/frame_diff_111.65s_3315.jpg
Saved output_differences/frame_diff_111.69s_3316.jpg
Saved output_differences/frame_diff_111.72s_3317.jpg
Saved output_differences/frame_diff_111.75s_3318.jpg
Saved output_differences/frame_diff_111.79s_3319.jpg
Saved output_differences/frame_diff_111.82s_3320.jpg
Saved output_differences/frame_diff_111.86s_3321.jpg
Saved output_differences/frame_diff_111.89s_3322.jpg
Saved output_differences/frame_diff_111.92s_33

Saved output_differences/frame_diff_114.31s_3394.jpg
Saved output_differences/frame_diff_114.35s_3395.jpg
Saved output_differences/frame_diff_114.38s_3396.jpg
Saved output_differences/frame_diff_114.42s_3397.jpg
Saved output_differences/frame_diff_114.45s_3398.jpg
Saved output_differences/frame_diff_114.48s_3399.jpg
Saved output_differences/frame_diff_114.52s_3400.jpg
Saved output_differences/frame_diff_114.55s_3401.jpg
Saved output_differences/frame_diff_114.58s_3402.jpg
Saved output_differences/frame_diff_114.62s_3403.jpg
Saved output_differences/frame_diff_114.65s_3404.jpg
Saved output_differences/frame_diff_114.69s_3405.jpg
Saved output_differences/frame_diff_114.72s_3406.jpg
Saved output_differences/frame_diff_114.75s_3407.jpg
Saved output_differences/frame_diff_114.79s_3408.jpg
Saved output_differences/frame_diff_114.82s_3409.jpg
Saved output_differences/frame_diff_114.85s_3410.jpg
Saved output_differences/frame_diff_114.89s_3411.jpg
Saved output_differences/frame_diff_114.92s_34

Saved output_differences/frame_diff_118.79s_3527.jpg
Saved output_differences/frame_diff_118.83s_3528.jpg
Saved output_differences/frame_diff_118.86s_3529.jpg
Saved output_differences/frame_diff_118.90s_3530.jpg
Saved output_differences/frame_diff_118.93s_3531.jpg
Saved output_differences/frame_diff_118.96s_3532.jpg
Saved output_differences/frame_diff_119.00s_3533.jpg
Saved output_differences/frame_diff_119.03s_3534.jpg
Saved output_differences/frame_diff_119.06s_3535.jpg
Saved output_differences/frame_diff_119.10s_3536.jpg
Saved output_differences/frame_diff_119.13s_3537.jpg
Saved output_differences/frame_diff_119.16s_3538.jpg
Saved output_differences/frame_diff_119.20s_3539.jpg
Saved output_differences/frame_diff_119.23s_3540.jpg
Saved output_differences/frame_diff_119.27s_3541.jpg
Saved output_differences/frame_diff_119.30s_3542.jpg
Saved output_differences/frame_diff_119.33s_3543.jpg
Saved output_differences/frame_diff_119.37s_3544.jpg
Saved output_differences/frame_diff_119.40s_35

Saved output_differences/frame_diff_121.79s_3616.jpg
Saved output_differences/frame_diff_121.83s_3617.jpg
Saved output_differences/frame_diff_121.86s_3618.jpg
Saved output_differences/frame_diff_121.89s_3619.jpg
Saved output_differences/frame_diff_121.93s_3620.jpg
Saved output_differences/frame_diff_121.96s_3621.jpg
Saved output_differences/frame_diff_121.99s_3622.jpg
Saved output_differences/frame_diff_122.03s_3623.jpg
Saved output_differences/frame_diff_122.06s_3624.jpg
Saved output_differences/frame_diff_122.09s_3625.jpg
Saved output_differences/frame_diff_122.13s_3626.jpg
Saved output_differences/frame_diff_122.16s_3627.jpg
Saved output_differences/frame_diff_122.20s_3628.jpg
Saved output_differences/frame_diff_122.23s_3629.jpg
Saved output_differences/frame_diff_122.26s_3630.jpg
Saved output_differences/frame_diff_122.30s_3631.jpg
Saved output_differences/frame_diff_122.33s_3632.jpg
Saved output_differences/frame_diff_122.36s_3633.jpg
Saved output_differences/frame_diff_122.40s_36

[out#0/rawvideo @ 0x7fcbb672ac40] video:3753787KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.000000%
frame= 3741 fps=206 q=-0.0 Lsize= 3753787KiB time=00:02:06.00 bitrate=244051.8kbits/s speed=6.93x    


In [31]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import os
import json
from sklearn.model_selection import train_test_split

# Step 1: Load Preprocessed Frames

def load_frames(output_folder):
    frames = []
    frame_timestamps = []
    frame_files = sorted(os.listdir(output_folder))  # Ensure frames are sorted by time

    for file in frame_files:
        if file.endswith(".jpg"):
            frame_path = os.path.join(output_folder, file)
            frame = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)
            height, width = frame.shape

            # Split the frame into top and bottom halves and stack them
            top_half = frame[:height // 2, :]
            bottom_half = frame[height // 2:, :]
            stacked_frame = np.vstack((top_half, bottom_half))

            frames.append(stacked_frame)

            # Extract timestamp from filename
            timestamp = float(file.split("_")[2][:-1])  # Extract the seconds value (e.g., 0.24s)
            frame_timestamps.append(timestamp)

    return np.array(frames), np.array(frame_timestamps)

# Step 2: Resize Frames and Save to Folder

def save_resized_frames(frames, target_size, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for idx, frame in enumerate(frames):
        resized_frame = cv2.resize(frame, target_size)
        output_path = os.path.join(output_folder, f"frame_{idx:04d}.jpg")
        cv2.imwrite(output_path, resized_frame)

# Step 3: Load JSON Data and Synchronize

def load_labels(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    label_timestamps = [entry['timestamp'] for entry in data['piano_vectors']]
    vectors = [entry['vector'] for entry in data['piano_vectors']]
    return np.array(label_timestamps), np.array(vectors)


def synchronize_data(frames, frame_timestamps, label_timestamps, vectors):
    synchronized_frames = []
    synchronized_vectors = []

    for label_timestamp in label_timestamps:
        # Find the three closest frame timestamps
        closest_indices = np.argsort(np.abs(frame_timestamps - label_timestamp))[:3]

        # Stack the three closest frames as channels
        selected_frames = np.stack([frames[idx] for idx in closest_indices], axis=-1)

        # Append the synchronized data
        synchronized_frames.append(selected_frames)
        synchronized_vectors.append(vectors[np.argmin(np.abs(label_timestamps - label_timestamp))])

    return np.array(synchronized_frames), np.array(synchronized_vectors)

# Step 4: Prepare Dataset

class PianoDataset(Dataset):
    def __init__(self, frames, vectors, target_size=(64, 64)):
        resized_frames = [cv2.resize(frame, target_size) for frame in frames]
        self.frames = torch.tensor(resized_frames, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0  # Normalize and rearrange
        self.vectors = torch.tensor(vectors, dtype=torch.float32)

    def __len__(self):
        return len(self.frames)

    def __getitem__(self, idx):
        return self.frames[idx], self.vectors[idx]

# Step 5: Define CNN Model

class PianoCNN(nn.Module):
    def __init__(self, input_channels, output_dim):
        super(PianoCNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        # Calculate input size dynamically
        dummy_input = torch.zeros(1, input_channels, 64, 64)  # Assuming resized input to 64x64
        with torch.no_grad():
            dummy_output = self.pool2(self.pool1(self.conv2(self.pool1(self.conv1(dummy_input)))))
        fc1_input_dim = dummy_output.numel()

        self.fc1 = nn.Linear(fc1_input_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Step 6: Train and Evaluate Model

def train_model(model, dataloader, criterion, optimizer, device, epochs=10):
    model = model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader)}")

# Example Usage
if __name__ == "__main__":
    # Paths to output folder and JSON file
    output_folder = "output_differences"
    json_path = "output.json"
    resized_folder = "cnn_input"

    # Load preprocessed frames
    frames, frame_timestamps = load_frames(output_folder)
    frame_height, frame_width = frames[0].shape[:2]  # Automatically detect frame dimensions

    # Save resized frames
    save_resized_frames(frames, target_size=(64, 64), output_folder=resized_folder)

    # Load labels and synchronize
    label_timestamps, vectors = load_labels(json_path)
    synchronized_frames, synchronized_vectors = synchronize_data(frames, frame_timestamps, label_timestamps, vectors)

    # Prepare dataset
    dataset = PianoDataset(synchronized_frames, synchronized_vectors, target_size=(64, 64))
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Define model, criterion, and optimizer
    input_channels = 3  # Three stacked frames
    output_dim = synchronized_vectors.shape[1]
    model = PianoCNN(input_channels, output_dim)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_model(model, dataloader, criterion, optimizer, device, epochs=10)

    # Save the model
    torch.save(model.state_dict(), "piano_key_model.pth")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x12544 and 3136x128)